# 검증세트

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH',]].to_numpy()
target = wine[['class']].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target,test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 훈련세트에 과대적합됨 

0.9971133028626413
0.864423076923077


# 교차검증

In [9]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.0059967 , 0.00599575, 0.00621223, 0.00701618, 0.0060041 ]),
 'score_time': array([0.00100493, 0.00100136, 0.00100327, 0.00099444, 0.00199437]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [10]:
import numpy as np
np.mean(scores['test_score'])

0.855300214703487

In [11]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

In [12]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

# 하이퍼파라미터 튜닝

In [13]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid= params,
    n_jobs=-1
)

In [14]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9615162593804117

In [16]:
# 매개변수 확인
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [17]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [19]:
params = {
    'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
    'max_depth' : range(5, 20, 1),
    'min_samples_split' : range(2, 100, 10)
}

gs = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid= params,
    n_jobs=-1    
)
gs.fit(train_input, train_target)

c:\Users\user\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [21]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

# 랜덤서치

In [24]:
# 매개 변수의 값이 수치일 때 범위나 간격을 미리 정하기 어려움

from scipy.stats import uniform, randint
# 균등분포에서 샘플링, randint = 정수, uniform= 실수
rgen = randint(0,10)
rgen.rvs(10)

array([1, 1, 4, 0, 4, 9, 1, 9, 4, 5], dtype=int64)

In [28]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 98, 107,  95,  97,  90,  99, 104, 101, 110,  99], dtype=int64))

In [30]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.81002706, 0.09752179, 0.1276052 , 0.55218889, 0.82074549,
       0.18092719, 0.55006772, 0.33098561, 0.2213293 , 0.40738819])

In [33]:
params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001),
    'max_depth' : randint(20,50),
    'min_samples_split' : randint(2,25),
    'min_samples_leaf' : randint(1,25)
}

In [34]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_distributions=params,
    n_iter=100,
    n_jobs=-1,
    random_state=42
)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022A846A1880>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022A844AC890>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022A846A0F20>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022A846A1F70>},
                   random_state=42)

In [35]:
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [36]:
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

In [37]:
dt = gs.best_estimator_
dt.score(test_input,test_target)

0.86